https://www.kaggle.com/thedownhill/art-images-drawings-painting-sculpture-engraving/version/2

In [1]:
import os
from keras import applications
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.models import Model, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Average
from tqdm import tqdm

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
def resnet50(model_input, num_classes):
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)
    for layer in base_model.layers:
        layer.trainable = False
    
    last = base_model.output
    x = Flatten()(last)
    x = Dense(15, activation='relu')(x)
    preds = Dense(num_classes, activation='softmax')(x)
    model = Model(base_model.input, preds)
    return model

def vgg16(model_input, num_classes):
    base_model = applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=model_input)
    last = base_model.output
    x = Flatten()(last)
    x = Dense(15, activation='relu')(x)
    preds = Dense(num_classes, activation='softmax')(x)
    model = Model(base_model.input, preds)
    return model

In [20]:
model_input = Input(shape=(224, 224, 3))
num_classes = 5
resnet50_model = resnet50(model_input, num_classes)
#vgg16_model = vgg16(model_input, num_classes)

In [21]:
base_path = r'D:/Jupyter/art-images-drawings-painting-sculpture-engraving/dataset/dataset_updated/'
#base_path = r'D:/Jupyter/art-images-drawings-painting-sculpture-engraving/dataset_small'
#train_data_dir = base_path + 'training_set'
train_data_dir = 'D:/Jupyter/art-images-drawings-painting-sculpture-engraving/dataset_small'
validation_data_dir = base_path + 'validation_set'
validation_data_dir = train_data_dir
test_data_dir = r'D:/Jupyter/art-images-drawings-painting-sculpture-engraving/musemart/dataset_updated/validation_set/painting'

In [22]:
img_height, img_width = 224, 224
batch_size = 100

In [23]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
train_data_dir, shuffle=False,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = valid_datagen.flow_from_directory(
validation_data_dir, shuffle=False,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 45 images belonging to 5 classes.
Found 45 images belonging to 5 classes.


In [24]:
steps_per_epoch = 3

In [25]:
resnet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
              metrics=['accuracy'])

In [26]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_148 (Activation)     (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

bn3a_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_158 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_158[0][0]             
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_159 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2b[0][0]              
__________________________________________________________________________________________________
res3a_bran

                                                                 activation_166[0][0]             
__________________________________________________________________________________________________
activation_169 (Activation)     (None, 28, 28, 512)  0           add_55[0][0]                     
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 14, 14, 256)  131328      activation_169[0][0]             
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_170 (Activation)     (None, 14, 14, 256)  0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_bran

__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_180[0][0]             
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_59 (Add)                    (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
                                                                 activation_178[0][0]             
__________________________________________________________________________________________________
activation_181 (Activation)     (None, 14, 14, 1024) 0           add_59[0][0]                     
__________________________________________________________________________________________________
res4e_bran

__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_191[0][0]             
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_192 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_192[0][0]             
__________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5b_branch2c[0][0]             
__________

In [27]:
resnet50_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, 
                             epochs=1, verbose=2, workers=12, validation_data=validation_generator)

Epoch 1/1
 - 93s - loss: 1.3872 - acc: 0.4889 - val_loss: 1.0270 - val_acc: 0.5333


In [28]:
train_features = resnet50_model.predict_generator(train_generator, workers=16)
valid_features = resnet50_model.predict_generator(validation_generator, workers=16)

In [29]:
train_features.shape

(45, 5)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='dogvgg16.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
model.fit(train_vgg16, train_targets, epochs=20, validation_data=(valid_vgg16, valid_targets), 
          callbacks=[checkpointer], verbose=1, shuffle=True)

In [ ]:
# get index of predicted dog breed for each image in test set
vgg16_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) 
                     for feature in test_vgg16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(vgg16_predictions)==
                           np.argmax(test_targets, axis=1))/len(vgg16_predictions)
print('\nTest accuracy: %.4f%%' % test_accuracy)

In [10]:
ensembled_models = [resnet50_model,vgg16_model]
def ensemble(models,model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(model_input,y,name='ensemble')
    return model

model_input = Input(shape=(224,224,3))
ensemble_model = ensemble(ensembled_models,model_input)

RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_4:0", shape=(?, 224, 224, 3), dtype=float32) at layer "input_4". The following previous layers were accessed without issue: []

-----